In [27]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import re
import emoji
import numpy as np
import os as os
import pandas as pd
import pickle

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preprocessing 
Removal of:
- emojis
- #'s
- @'s
- links


In [108]:
# dataset = pd.read_csv('twitter_data/tweets25k.csv')

# # first 3 elements
# print(dataset.head(3))

# # removing #'s, @'s, http's and emojis
# def emoji_free_text(text):
#     text.encode('ascii','ignore').decode('ascii')
#     text = re.sub(emoji.get_emoji_regexp(), r'', text)
#     return text
         
# for i in range(0,len(dataset)):
#     dataset.iloc[i,1] = re.sub(r"@\S+","",str(dataset.iloc[i,1]))
#     dataset.iloc[i,1] = re.sub(r"http\S+","", dataset.iloc[i,1])
#     dataset.iloc[i,1] = re.sub(r"#\S+","", dataset.iloc[i,1])
#     dataset.iloc[i,1] = emoji_free_text(dataset.iloc[i,1])
# print("---------------------------")
# # first 3 elements after removal
# print(dataset.head(3))
# dataset.to_pickle('twitter_processed.pkl')

   label                                               text
0      1  @GodNegus: #IHatePokemonGoBecause i have not s...
1      1  @LibertyNow10: .@SpeakerRyan when you come up ...
2      1  John Podesta Trends on Facebook After Peter Sc...
---------------------------
   label                                               text
0      1            i have not seen one wild charmander yet
1      1   . when you come up with better solution then ...
2      1  John Podesta Trends on Facebook After Peter Sc...


# Each encoded_layers in encoded_layers_list has 

1) The layer number (12 layers)

2) The batch number (1 sentence)

3) The word / token number (n tokens in our sentence)

4) The hidden unit / feature number (768 features)


In [121]:
%%capture
# Load dataset
dataset = pd.read_pickle("twitter_processed.pkl") 
# Load pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [152]:
sentence_embeddings_list = []

for tweet in dataset.text.to_list():
    # Add the special tokens.
    marked_text = "[CLS] "+ tweet + " [SEP]"
    # Split the sentence into tokens.
    tokenized_text = tokenizer.tokenize(marked_text)
    # Map the token strings to their BERT vocabulary indices.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # for single-sentence input mark each token as belonging to sentence "1".
    segments_ids = [1]*len(tokenized_text)
    # converting to pytorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor,segments_tensors)
    # Each layer in encoded_layers list is a torch tensor where 
    # shape for each of the 12 layers:  torch.Size([1, num_tokens, 768])
    # we will be averaging the second to last hidden layer of each token to get a single
    # [1,768] size vector as the sentence embedding (experiment with this)
    token_vecs = encoded_layers[11][0]
    # Calculate the average of all token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
    sentence_embeddings_list.append(sentence_embedding)

In [153]:
#pickle data for use in model
with open('tweet_embeddings25k.pkl', 'wb') as f:
    pickle.dump(sentence_embeddings_list, f)
#pickle labels for later use
with open('labels25k.pkl', 'wb') as f:
    pickle.dump(dataset['label'], f)